In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
!pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


In [8]:
import os
import pandas as pd
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # Replace with your chosen model
from sklearn.metrics import accuracy_score
import joblib
from io import BytesIO
from pydub import AudioSegment
import wave
import math
import uuid

# Data paths (replace with your actual file paths)
cry_data_path = r"C:\Users\acer\Documents\Cry\cry\Cry"
non_cry_data_path = r"C:\Users\acer\Documents\Cry\cry\Not Cry"

# Feature extraction function
def extract_features(audio_file, max_length=100):
    audiofile, sr = librosa.load(audio_file)
    fingerprint = librosa.feature.mfcc(y=audiofile, sr=sr, n_mfcc= 2000)
    if fingerprint.shape[1] < max_length:
        pad_width = max_length - fingerprint.shape[1]
        fingerprint_padded = np.pad(fingerprint, pad_width=((0, 0), (0, pad_width)), mode='constant')
        return fingerprint_padded.T
    elif fingerprint.shape[1] > max_length:
        return fingerprint[:, :max_length].T
    else:
        return fingerprint.T

# Load data and extract features
X = []  # Feature vectors
y = []  # Labels
max_length = 100

# Extract features from cry recordings
for filename in os.listdir(cry_data_path):
    filepath = os.path.join(cry_data_path, filename)
    features = extract_features(filepath)
    X.append(features)
    y.append(1)  # Label as cry (1)

# Extract features from non-cry recordings
for filename in os.listdir(non_cry_data_path):
    filepath = os.path.join(non_cry_data_path, filename)
    features = extract_features(filepath)
    X.append(features)
    y.append(0)  # Label as non-cry (0)


c:\Users\acer\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [9]:

# Convert to numpy arrays
X = np.asarray(X)
y = np.asarray(y)

# Flatten the features and labels
X_flat = X.reshape(X.shape[0], -1)
y_flat = y

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flat, y_flat, test_size=0.2)

# Train the model (replace with your chosen model)
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Save the model
model_path = r"C:\Users\acer\Documents\Cry\cry\cry.pkl"
joblib.dump(model, model_path)
print("Model saved to:", model_path)

Accuracy: 0.941952506596306
Model saved to: C:\Users\acer\Documents\Cry\cry\cry.pkl


# Prediction Test

In [10]:
# Function to predict cry for new audio
def predict_cry(audio_path, model_path=model_path):
    # Load and pre-process the audio
    y, sr = librosa.load(audio_path)
    # Extract features
    features = extract_features(audio_path)
    # Reshape features if necessary
    features = features.reshape(1, -1)
    # Load the trained model
    model = joblib.load(model_path)
    # Make prediction
    prediction = model.predict(features)
    if prediction[0] == 1:
        print("Cry detected!")
    else:
        print("No cry detected.")

# Example usage
new_audio_path = "cry.wav"
predict_cry(new_audio_path)

C:\Users\acer\AppData\Local\Temp\ipykernel_10236\117697957.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path)
c:\Users\acer\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'cry.wav'